# Второй стартер

Начнем "копать" поглубже в признаках автомобилей 🏎. Создадим модель лучшего качества. Ну и, конечно, начнем вникать в наиболее интересные признаки, чтобы потом еще больше улучшить качество модели (создавая новые признаки).

## Импортируем библиотеки

In [ ]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np
import helper as h

from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.model_selection import cross_val_score
import eli5
from tqdm import tqdm

import gc

## Загружаем данные

Сейчас уже сразу импортируем и `train`, и `test`. Почему? 
Потому что дальше будем создавать совместные признаки, например, используя `.factorize()`. Кстати, подумай, что произойдет плохого, если запустим `.factorize()` отдельно для `train` и `test`? 

In [ ]:
df_train = pd.read_hdf("../input/df.train.h5")
df_train['price_value'] = df_train['price_value'].map(h.parse_price)

df_test = pd.read_hdf("../input/df.test.h5")

print(df_train.shape, df_test.shape)

## [Выброс](https://ru.wikipedia.org/wiki/%D0%92%D1%8B%D0%B1%D1%80%D0%BE%D1%81_(%D1%81%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0)) (англ. `outlier`)

Давай проверим машины, которые дороже 3 млн польских злотых (примерно $1млн).

In [ ]:
df_train[ df_train["price_value"] > 3000_000 ]

Странно очень, что `Honda` стоит 9_999_999_999_999. Здесь явно произошла какая-та ошибка, поэтому давай сразу это удалим, иначе у нас "сломается" среднее значение.

In [ ]:
df_train = df_train[ df_train.index != 106447 ]

## ☝️ Обрати внимание
Обычно мы не удаляем данные. Но все же здравый смысл должен присутствовать. Если видим, что есть что-то совсем аномальное (например, цена за авто больше, чем ВВП Польши, то это как-то странно 😂). Главное запомни, что никогда не удаляй данные, иначе можно потерять что-то важное. Затем вспомни, что никогда не говори „никогда“. Иногда бывают исключения. Но для того они и исключения, что появляются очень редко. Вот сейчас такой случай. Обрати внимание, что мы не удаляли, например, 1% данных, т.к. скорее все это может навредить (потеряем информацию, ведь это реальный случай и дорогие машины тоже существуют).

## Объединяем
Соединяем `df_train` и `df_test` и записываем результат в `df`.


### ⌛️⌛️⌛️ Внимание ☝️
У нас достаточно много данных, поэтому спокойно подожди пока проходит трансформация из словаря в `dataframe` и потом объединение данных с главным `df`.

In [ ]:
df = pd.concat([df_train, df_test])
df.shape

Можем посмотреть 👀  на 5 случайных строк.

In [ ]:
df.sample(5)

## `offer_params`
Здесь куча полезной информации про автомобили, поэтому с этого и начнем. Нам нужно "распаковать словарик" в табличку (`dataframe`). Здесь очень пригодится простая, но надежная структура.

In [ ]:
df["offer_params"].head().apply(pd.Series)

In [ ]:
params = df["offer_params"].apply(pd.Series)
params = params.fillna(-1)

if "Bezwypadkowy" not in df:
    df = pd.concat([df, params], axis=1)
df.shape

Кстати, условие `if "Bezwypadkowy" not in df` защищает нас от добавления `params` к `df` еще раз, в случае запуска ячейки более 1 раза. Такое может произойти, поэтому стоит писать код таким образом, чтобы, запуская ячейку более одного раза, результат остался прежний. Этот простой навык поможет сберечь кучу нервов 😂.


## ☝️ Возможно, пугает, что не совсем понятно - о чем речь, ведь здесь все по-польски?

Давай помогу перевести. Хотя это легко делается. Копируется список названий столбцов по-польски (например, `params.columns`) в переводчик и потом создаем такой-же на русском языке и затем словарик, ключ - польские слова и значения по-русски. Нам это пригодится на следующем 
шаге. 

Обрати внимание, что в `params` также есть значения и на английском, их пока переводить не будем (но если нужно - можно перевести 😉).

In [ ]:
ru_params = ["Безаварийный", "Количество мест", "Страна происхождения",
       "Объем", "Тип топлива", "Коробка передач",
       "Кредит", "Первый владелец", "Предложение от",
       "Количество дверей", "Пробег", "Мощность", "Металлик", "Тип", "Цвет",
       "Модель автомобиля", "Состояние", "Категория", "Обслуживание в сервисном центре", "Привод",
       "Лизинг", "Год выпуска", "Марка автомобиля", "Счет-фактура по НДС",
       "Первая регистрация", "Зарегистрировано в Польше", "Маржа НДС",
       "Версия", "VIN", "Перламутр", "Поврежденный", "Код двигателя",
       "Сажевый фильтр", "Выбросы CO2", "Ежемесячный платеж",
       "Количество оставшихся платежей", "Первоначальный платеж", "Сумма погашения",
       "Акрил (неметаллический)", "Тюнинг", "Правый руль (английский)",
       "Допуск грузовика"]


dict_params = {pl:ru for pl,ru in zip(params.columns, ru_params)}
dict_params

Теперь можно поменять названия в данных в столбцах. Это легко сделать при помощи `.rename()`.

In [ ]:
df.columns

In [ ]:
df.rename(columns=dict_params, inplace=True)
df

## Сколько уникальных значений?

Можно посмотреть, сколько уникальных значений в каждом признаке. В коде не нужно особо разбираться, лучше посмотри на результат.

In [ ]:
feats_nunique = {feat:df[feat].nunique() for feat in ru_params}
feats_nunique

In [ ]:
obj_feats = df.select_dtypes(object).columns

for feat in tqdm(obj_feats):
    some_value = df[feat].values[0]
    if isinstance(some_value, list): continue
    if isinstance(some_value, dict): continue
    if "price" in feat: continue
        
    df["{}_cat".format(feat)] = df[feat].factorize()[0]
    
cat_feats = [x for x in df.columns if "_cat" in x]
cat_feats

Теперь, когда признаки созданы и для `test`, и для `train`, можно возвращать разделение данных и будем уже обучать модель и прогнозировать.


А как разделить? Применим простой трюк, ведь в тестовых данных не было цены (`price_value`), поэтому можем сделать простой фильтр: если цена есть - то это `train` в противном случае - `test`.

In [ ]:
df_train = df[ ~df["price_value"].isnull() ].copy()
df_test = df[ df["price_value"].isnull() ].copy()

df_train.shape, df_test.shape

## X, y

Теперь подготавливаем матрицу с признаками (сразу `X_train` и  `X_test`) и наши ответы (`y_train`).

In [ ]:
X_train = df_train[cat_feats]
y_train = df_train["price_value"]

X_test = df_test[cat_feats]

## Валидация модели
Проверяем локально качество модели.
Знак минус - это техническая деталь, проигнорируй.

Но если все же оооочень интересно - "а почему так". То вот пару слов с объяснениями:
просто разные метрики стремятся в разные стороны, некоторые к нулю, а некоторые к единице. Поэтому чтобы максимизировать `Score` во всех случаях (так код "абстрактно" написан), для метрик стремящихся к нулю, добавили знак минус (т.е. самое большое возможное значение будет 0).

In [ ]:
model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")
np.mean(scores), np.std(scores)

## Тренируем модель и прогнозируем

Для начала возьмем простую модель - дерево решений.

In [ ]:
model = DecisionTreeRegressor(max_depth=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Записываем результат

И отправляем в Kaggle. Это должно дать около `score=~21k`. Кстати, помни, что количество `submit` на Kaggle ограничено - только 5 штук в день. Поэтому учись проверять результат локально. Он не должен идеально совпадать. Но должна быть взаимосвязь (локальные улучшения заметны на **Public Leaderboard**).

In [ ]:
df_test["price_value"] = y_pred
df_test[ ["id", "price_value"] ].to_csv("../output/decision_tree.csv", index=False)

## Анализируем


Используя библиотеку `eli5`, можно попробовать посмотреть, какие признаки наиболее "интересны" для модели (повлияли на результат).

### ☝️Осторожно!
Нельзя к этому рэнкингу относиться сильно "доверчиво". Очень много подводных камней, которые нужно уметь правильно интерпретировать. Поэтому отнесись к этому просто с уважением и предположением, что, наверное, стоит посмотреть глазами 👀 и пощупать руками💪.

In [ ]:
eli5.show_weights(model, feature_names=cat_feats)

Давай покажу, как можно проанализировать все то, что уже видно.

Например, видно что **"Привод"** - важный признак. Давай посмотрим, как изменяется цена.

In [ ]:
pd.pivot_table(df, index=["Привод"], values=["price_value"], aggfunc=["mean", "median", "min", "max", len])

Можно сделать разные выводы, можем начать вместе:
- на переднеприводные автомобили медиана цены - **21 999**, а на заднеприводные - **29 990**.
- для полноприводных (4x4 (stały)) медина -  **92 900** злотых (дороже, более, чем в 3 раза, по сравнению с переднеприводными или заднеприводными!) и таких автомобилей больше **10k+**
- ...

### 🧐 Какие еще выводы можно сделать? 

Поделись, пожалуйста, своими размышлениями в [#dwthon_2_ideas](https://dataworkshop-ru.slack.com/archives/C02FG4J7TU4) 🤝.

## Мощность

Посмотрим аналогичным образом.

In [ ]:
pd.pivot_table(df, index=["Мощность"], values=["price_value"], aggfunc=["mean", "median", "min", "max", len])

Кстати, здесь KM в переводе обозначает - лошадиные силы. Хорошо было бы сделать нормализацию и вытянуть `int` или `float`.

## Примечание 

Понятие **«лошадиная сила автомобиля»** 🐎 было введено ещё в 18 веке Джеймсом Уаттом. Это параметр, показывающий мощность автомобиля, сравниваемую с силой лошади.

1 лошадиная сила или л.с. равна мощности, необходимой для подъёма **75-килограммового** груза на высоту один метр за **1 секунду**. В некоторых случаях принято переводить л.с. в киловатты — тогда 1 лошадиная сила будет равна **735,5 Вт** или **0,735 кВт**.

Для определения мощности в л.с. конкретного автомобиля, надо перевести кВт, указанные в паспортных данных, в лошадиные силы. Делается это так: приведённые значения в киловаттах просто делятся на **0,735**. Итоговое значение и будет означать лошадиные силы определённого автомобиля.

Как интересно 😉.

### 🧐 Какие выводы можно сделать? 

Поделись пожалуйста своимии размышлениями в [#dwthon_2_ideas](https://dataworkshop-ru.slack.com/archives/C02FG4J7TU4) 🤝.

## Что дальше?

Надеюсь уже задана "траектория мышления". Теперь нужно покопаться в данных (например, проверить 10-30 признаков) и сделать выводы (лучше всего эти выводы записать, наше внимание и мышление лучше работают, когда не нужно держать в голове много монотонной информации).


Обязательно поделись своими размышлениями в Slack - [#dwthon_2_ideas](https://dataworkshop-ru.slack.com/archives/C02FG4J7TU4) 🤝. Вместе можно выдумать больше.

Кстати, сегодня будет еще один стартер, но все же перед ним - поработай с теми идеями, которые появились в этом стартере, чтобы потом было на что опереться.